# Video Game Sales Data Analysis 
This Jupyter notebook will explore a dataset of international video game sales in order to identify patterns that determine whether a game succeeds or not.


#### Project Sections 
1. Initial Set Up and Data Preparation 
2. General Data Analysis 
3. Analysis By Sales Region 
4. Hypothesis Testing 

### Initial Set Up & Data Preparation 

#### Data Cleaning 
Upon initial exploration of the data, it has been determined that the following tasks must be carried out in order to prepare the data for analysis: 
 - 1
 - 2
 - 3

### General Data Analysis


### Analysis by Sales Region
In this section, comparisons will be made between the three sales regions: North America, Europe, and Japan. The following factors will be analyzed:
- Platform market share variations by region
- Genre popularity variation across regions 
- Effect ESRB rating by region 

#### Platform Market Share Variations Across Regions

#### Genre Popularity Variations Across Regions

#### ESRB Rating Effect by Region 

### Hypothesis Testing 
This section will utilize hypothesis testing to see if the following two hypotheses are supported through statistical analysis:
1. Average user ratings of the Xbox One and PC platforms are the same. 
2. Average user ratings for the Action and Sports genres are different.

#### Are the average user ratings of the Xbox One and PC platforms the same?
- Null Hypothesis: 
- Alternative Hypothesis: 

#### Are the average user ratings for the Action and Sports genres are different? 
- Null Hypothesis: 
- Alternative Hypothesis:



## Conclusions 